In [ ]:
!pip install nltk

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [6]:
from nltk.corpus import brown

# Training the HMM tagger
brown_tagged_sents = brown.tagged_sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

# Defining the HMM tagger
hmm_tagger = nltk.HiddenMarkovModelTagger.train(train_sents)

# Function to implement Viterbi algorithm
def viterbi(words):
    return hmm_tagger.tag(words)

In [7]:
# Test the Viterbi algorithm
test_sent = "This is John. One day he saw a saw and decided to take it to the dinning table".split()
res = viterbi(test_sent)
for ele in res:
  print(ele)

('This', 'DT')
('is', 'BEZ')
('John.', 'RB')
('One', 'CD')
('day', 'NN')
('he', 'PPS')
('saw', 'VBD')
('a', 'AT')
('saw', 'NN')
('and', 'CC')
('decided', 'VBD')
('to', 'TO')
('take', 'VB')
('it', 'PPO')
('to', 'IN')
('the', 'AT')
('dinning', 'JJ')
('table', 'NN')


In [9]:
from collections import defaultdict

# Define transition probabilities based on observed tags
transition_probabilities = {
    'DT': {'DT': 0, 'NN': 0.5, 'JJ': 0.5},
    'NN': {'DT': 0.2, 'NN': 0.2, 'VB': 0.6},
    'JJ': {'NN': 1.0},
    'RB': {'CD': 1.0},
    'CD': {'NN': 1.0},
    'PPS': {'VBD': 1.0},
    'VBD': {'AT': 0.5, 'JJ': 0.5},
    'AT': {'NN': 1.0},
    'CC': {'VBD': 1.0},
    'TO': {'VB': 1.0},
    'VB': {'PPO': 1.0},
    'PPO': {'IN': 1.0},
    'IN': {'AT': 1.0},
}

# Define starting probability
start_probabilities = {'DT': 1.0}

# Function to find the most probable sequence
def find_probable_sequence(tagged_words, transition_probabilities, start_probabilities):
    sequence = []
    current_tag = None

    # Initialize the current_tag based on starting probabilities
    max_start_prob = 0
    for tag, prob in start_probabilities.items():
        if prob > max_start_prob:
            max_start_prob = prob
            current_tag = tag

    sequence.append((tagged_words[0][0], current_tag))

    # Iterate over the remaining words
    for i in range(1, len(tagged_words)):
        prev_tag = current_tag
        current_word, _ = tagged_words[i]

        max_prob = 0
        for next_tag, prob in transition_probabilities[current_tag].items():
            if prob > max_prob:
                max_prob = prob
                current_tag = next_tag

        sequence.append((current_word, current_tag))

    return sequence

# Example tagged words
tagged_words = [('This', 'DT'), ('John.', 'RB'), ('One', 'CD'), ('is', 'BEZ'),('day', 'NN'), ('he', 'PPS'), ('saw', 'VBD'), ('a', 'AT'), ('saw', 'NN'), ('and', 'CC'), ('decided', 'VBD'), ('to', 'TO'), ('take', 'VB'), ('it', 'PPO'), ('to', 'IN'), ('the', 'AT'), ('dinning', 'JJ'), ('table', 'NN')]

# Find the most probable sequence
probable_sequence = find_probable_sequence(tagged_words, transition_probabilities, start_probabilities)

# Print the most probable sequence
for word, tag in probable_sequence:
    print(f'({word}, {tag})', end=' ')


(This, DT) (John., NN) (One, VB) (is, PPO) (day, IN) (he, AT) (saw, NN) (a, VB) (saw, PPO) (and, IN) (decided, AT) (to, NN) (take, VB) (it, PPO) (to, IN) (the, AT) (dinning, NN) (table, VB) 

In [17]:
import nltk
from nltk.corpus import brown
from nltk.tag import HiddenMarkovModelTagger

# Training the HMM tagger
brown_tagged_sents = brown.tagged_sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

# Defining the HMM tagger
hmm_tagger = HiddenMarkovModelTagger.train(train_sents)

# Function to implement Viterbi algorithm and get probabilities
def viterbi_with_prob(words):
    tagged_words = hmm_tagger.tag(words)
    probabilities = [prob for _, prob in tagged_words]
    return tagged_words, probabilities

# Example words
words_to_label = "This is John. One day he saw a saw and decided to take it to the dining table".split()

# Use Viterbi algorithm to label words and get probabilities
tagged_words, probabilities = viterbi_with_prob(words_to_label)

# Print the labeled sequence with associated probabilities
for (word, _), prob in zip(tagged_words, probabilities):
    print(f'({word}, {prob})', end=' ')


(This, DT) (is, BEZ) (John., RB) (One, CD) (day, NN) (he, PPS) (saw, VBD) (a, AT) (saw, NN) (and, CC) (decided, VBD) (to, TO) (take, VB) (it, PPO) (to, IN) (the, AT) (dining, NN) (table, NN) 

In [20]:
import nltk

# Download the Penn Treebank corpus
nltk.download('treebank')

# Load the Penn Treebank corpus
corpus = nltk.corpus.treebank.tagged_sents()

# Count transition probabilities from the corpus
transition_counts = {}
pos_counts = {}
for sentence in corpus:
    previous_tag = None
    for word, tag in sentence:
        if previous_tag is not None:
            if previous_tag not in transition_counts:
                transition_counts[previous_tag] = {}
            if tag not in transition_counts[previous_tag]:
                transition_counts[previous_tag][tag] = 0
            transition_counts[previous_tag][tag] += 1
        if tag not in pos_counts:
            pos_counts[tag] = 0
        pos_counts[tag] += 1
        previous_tag = tag

# Calculate transition probabilities
transition_probabilities = {}
for prev_tag, next_tags in transition_counts.items():
    transition_probabilities[prev_tag] = {}
    total_transitions = sum(next_tags.values())
    for next_tag, count in next_tags.items():
        transition_probabilities[prev_tag][next_tag] = count / total_transitions

# Define a simple POS tagger
def simple_pos_tagger(word):
    if word.lower() in ['the', 'a', 'an']:
        return 'DT'  # Determiner
    else:
        return 'NN'  # Noun (just for simplicity)

# Implement Viterbi algorithm
def viterbi(tokens, initial_probabilities, transition_probabilities, pos_tagger):
    # Initialize viterbi matrix
    viterbi_matrix = [{}]

    # Initialize backpointer
    backpointer = [{}]

    # Initialize first column of viterbi matrix
    for pos_tag in initial_probabilities:
        viterbi_matrix[0][pos_tag] = initial_probabilities[pos_tag] * transition_probabilities[pos_tag].get(pos_tagger(tokens[0]), 0)
        backpointer[0][pos_tag] = None

    # Fill in the viterbi matrix
    for t in range(1, len(tokens)):
        viterbi_matrix.append({})
        backpointer.append({})
        for pos_tag in initial_probabilities:
            max_prob = max(viterbi_matrix[t-1][prev_tag] * transition_probabilities[prev_tag].get(pos_tag, 0) for prev_tag in initial_probabilities)
            viterbi_matrix[t][pos_tag] = max_prob * transition_probabilities[pos_tag].get(pos_tagger(tokens[t]), 0)
            backpointer[t][pos_tag] = max(initial_probabilities.keys(), key=lambda prev_tag: viterbi_matrix[t-1][prev_tag] * transition_probabilities[prev_tag].get(pos_tag, 0))

    # Find the tag sequence with highest probability
    max_prob = max(viterbi_matrix[-1][pos_tag] for pos_tag in initial_probabilities)
    tag_sequence = []
    prev_tag = max(initial_probabilities.keys(), key=lambda pos_tag: viterbi_matrix[-1][pos_tag])
    for t in range(len(tokens)-1, -1, -1):
        tag_sequence.insert(0, prev_tag)
        prev_tag = backpointer[t][prev_tag]

    return tag_sequence

# Run Viterbi algorithm on an example sentence
sentence = "The quick brown fox jumps over the lazy dog"
tokens = nltk.word_tokenize(sentence)
initial_probabilities = {'DT': 0.5, 'NN': 0.5}  # Initial probabilities can be refined too
predicted_tags = viterbi(tokens, initial_probabilities, transition_probabilities, simple_pos_tagger)

# Print the result
print("Predicted POS tags:", predicted_tags)


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


Predicted POS tags: ['NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN', 'NN']
